En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
from scipy import stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [14]:
df = pd.read_pickle("datos/peliculas_limpio.pkl")
df.head(2)

,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Meta_score,No_of_Votes,Gross,Rating_BOX,Ryr_Sklstd,Rnt_Sklstd,Meta_Sklstd,Votes_Sklstd,Gross_Sklstd
0,1994.0,A,142,Drama,9.3,80.0,2343110.0,28341469.0,0.100485,-0.151515,0.676471,-0.001712,6.918768,0.131429
1,1972.0,A,175,Crime,9.2,100.0,1620367.0,134966411.0,0.100485,-0.818182,1.647059,1.426860,4.650520,1.997227


Vuestro set de datos debería tener al menos una variable categórica, el objetivo del pair programming de hoy:
-Hacer una códificación de la/las variables categóricas que tengáis en vuestro set de datos.
-Recordad que lo primero que deberéis hacer es decidir su vuestras variables tienen o no orden, para que en función de esto uséis una aproximación u otra.
-Guardad el dataframe, donde deberíais tener las variables estadandarizas, normalizadas y codificadas en un csv para usarlo en el próximo pairprogramming

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
# Vamos a darle un valor númerico a la variable categórica "Certificate"
le = LabelEncoder()
df['Certificate_le']=le.fit_transform(df['Certificate'])

In [17]:
df.sample(5)

,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Meta_score,No_of_Votes,Gross,Rating_BOX,Ryr_Sklstd,Rnt_Sklstd,Meta_Sklstd,Votes_Sklstd,Gross_Sklstd,Certificate_le
700,1967.0,U,108,Thriller,7.8,81.0,27733.0,1.755074e+07,0.100485,-0.969697,-0.323529,0.069717,-0.347782,-0.057395,12
96,2000.0,UA,104,Comedy,8.3,55.0,782001.0,3.032816e+07,0.100485,0.030303,-0.441176,-1.787426,2.019403,0.166193,14
647,2000.0,UA,143,Adventure,7.8,73.0,524235.0,2.336321e+08,0.100485,0.030303,0.705882,-0.501712,1.210434,3.723750,14
986,1978.0,U,91,Animation,7.6,64.0,33656.0,1.650431e+07,0.100485,-0.636364,-0.823529,-1.144569,-0.329193,-0.075706,12
768,2006.0,R,110,Action,7.7,53.0,299524.0,2.249449e+07,0.100485,0.212121,-0.264706,-1.930283,0.505203,0.029114,8
